# Data Science - Laboratorio 9
## Visualizaciones interactivas y dashboards
### Modelos LSTM
---
**Integrantes**
- Diego Alberto Leiva
- José Pablo Orellana

## Librerias

In [1]:
# Manipulacion de Datos
import pandas as pd
import numpy as np

# Sistema
import os

# Utilidades
import random

# Preprocesamiento
from sklearn.preprocessing import StandardScaler

# PyTorch CUDA
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

### Carga de Datos Limpios

In [2]:
# Leer los archivos de una carpeta y almacenarlos como dataframes
consumos = pd.read_csv('data/consumos_clean.csv', sep=';', encoding='utf-8')
importaciones = pd.read_csv('data/importaciones_clean.csv', sep=';', encoding='utf-8')
precios = pd.read_csv('data/precios_clean.csv', sep=';', encoding='utf-8')

In [3]:
consumos.shape

(293, 4)

## Modelos LSTM

### Configuracion de PyTorch con CUDA

Se verifica si el sistema tiene capacidad de CUDA o no

In [4]:
# Configurar la semilla para la reproducibilidad
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    print(f"CUDA AVAILABLE")

    print(f"Using: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    # Configurar el generador de números aleatorios de CUDA
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU.

    # Configurar PyTorch a determinista
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
else:
    # Si no hay GPU disponible, usar la CPU
    print("CUDA NOT AVAILABLE")
    print("Using CPU")

CUDA AVAILABLE
Using: NVIDIA GeForce RTX 3070 Ti Laptop GPU


### Arquitectura de una LSTM con PyTorch CUDA

In [5]:
class LSTM_Model():
    """
    Clase para entrenar un modelo LSTM para predecir series temporales y actualizar el dataframe con las predicciones.

    Args:
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.

    Attributes:
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.
        model (torch.nn.Module): Modelo LSTM.
        loss_fn (torch.nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        train_size (int): Tamaño del conjunto de entrenamiento.
    """
    def __init__(self, hidden_layers, num_layers, lookback, n_epochs, test_size, dataframe, target_column):
        
        # Parámetros fijos
        self.input_size = 1                    # Fijo, tamaño de entrada
        self.output_size = 1                   # Fijo, tamaño de salida
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Dispositivo fijo
        
        # Inicializar otros hiperparámetros
        self.hidden_layers = hidden_layers     # Número de capas ocultas
        self.num_layers = num_layers           # Número de capas LSTM
        self.lookback = lookback               # Número de pasos de tiempo hacia atrás
        self.n_epochs = n_epochs               # Número de épocas
        self.test_size = test_size             # Tamaño del conjunto de prueba
        self.dataframe = dataframe             # Dataframe de entrada
        self.target_column = target_column     # Columna objetivo

        # Crear el modelo LSTM 
        self.model = self.build_lstm_model()

        # Definir la función de pérdida y el optimizador
        self.loss_fn = nn.MSELoss().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        # Entrenar el modelo y actualizar el dataframe
        self.train_preds, self.test_preds = self.train_model(dataframe, target_column)
        self.updated_data = self.add_preds(self.dataframe, target_column, self.train_preds, self.test_preds)

    def build_lstm_model(self):
        """
        Función para construir el modelo LSTM.

        Returns:
            model (torch.nn.Module): El modelo LSTM.
        """
        class LSTM(nn.Module):
            def __init__(self, input_size, hidden_layers, output_size, num_layers):
                super(LSTM, self).__init__()

                # Definir la capa LSTM
                self.lstm = nn.LSTM(input_size=input_size, 
                                    hidden_size=hidden_layers, 
                                    num_layers=num_layers, 
                                    batch_first=True)

                # Definir la capa de salida
                self.linear = nn.Linear(hidden_layers, output_size)

                # Capa de activación ReLU
                self.relu = nn.ReLU()
                
            def forward(self, x):
                # Inicializar hidden states
                h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
                c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

                # Forward pass a través de LSTM
                out, _ = self.lstm(x, (h0, c0))

                # Aplicar la capa de activación
                out = self.relu(out)

                # Solo el último hidden state
                out = self.linear(out[:, -1, :])
                
                # Devolver la predicción
                return out

        # Crear el modelo LSTM con los hiperparámetros definidos
        return LSTM(self.input_size, self.hidden_layers, self.output_size, self.num_layers).to(self.device)


    def prepare_data(self, dataset):
        """
        Función para crear el dataset de entrenamiento.

        Args:
            dataset (numpy.array): El dataset de entrada.

        Returns:
            X (torch.Tensor): El tensor de features.
            y (torch.Tensor): El tensor de target.
        """
        X, y = [], []
        for i in range(len(dataset) - self.lookback):
            X.append(dataset[i:i + self.lookback])
            y.append(dataset[i + self.lookback])

        # Convertir listas a numpy arrays y luego a tensores
        X = np.array(X)
        y = np.array(y)

        # Retornar tensores
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


    def train_model(self, dataframe, variable):
        """
        Función para entrenar el modelo LSTM.

        Args:
            dataframe (pandas.DataFrame): El dataframe de entrada.

        Returns:
            pred_train (numpy.array): Predicciones sobre el conjunto de entrenamiento.
            pred_test (numpy.array): Predicciones sobre el conjunto de prueba.
        """
        # Preparar los datos
        df = dataframe.copy()
        timeseries = df[[variable]].values.astype(float)

        # Normalizar los datos
        scaler = StandardScaler()
        timeseries = scaler.fit_transform(timeseries)

        # Dividir en entrenamiento y prueba
        self.train_size = len(timeseries) - self.test_size
        train_data = timeseries[:self.train_size]
        test_data = timeseries[self.train_size:]

        # Crear datasets
        X_train, y_train = self.prepare_data(train_data)
        X_test, y_test = self.prepare_data(test_data)

        # Mover los tensores al dispositivo
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        X_test, y_test = X_test.to(self.device), y_test.to(self.device)

        # Crear DataLoader para entrenamiento
        loader = data.DataLoader(data.TensorDataset(X_train, y_train), batch_size=16, shuffle=False)

        # Entrenar el modelo
        for epoch in range(self.n_epochs):
            self.model.train()
            for X_batch, y_batch in loader:
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

                # Predicción
                y_pred = self.model(X_batch)

                # Calcular pérdida
                loss = self.loss_fn(y_pred, y_batch)

                # Backpropagation y optimización
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

        # Predicciones sobre el conjunto de entrenamiento y prueba
        self.model.eval()
        with torch.no_grad():
            pred_train = self.model(X_train).cpu().numpy()
            pred_test = self.model(X_test).cpu().numpy()

        # Desnormalizar las predicciones 
        pred_train = scaler.inverse_transform(pred_train)
        pred_test = scaler.inverse_transform(pred_test)
        
        # Retornar predicciones
        return pred_train, pred_test

    def add_preds(self, df, target_column, train_preds, test_preds):
        """
        Agrega columnas de predicciones de entrenamiento y prueba al dataframe base tomando en cuenta el lookback.

        Args:
            df (pandas.DataFrame): Dataframe original.
            target_column (str): Columna objetivo para las predicciones.
            train_preds (numpy.array): Predicciones del conjunto de entrenamiento.
            test_preds (numpy.array): Predicciones del conjunto de prueba.

        Returns:
            df (pandas.DataFrame): Dataframe actualizado con las predicciones.
        """
        col_pred_train = f'pred_train_{target_column}'
        col_pred_test = f'pred_test_{target_column}'

        # Agregar columna de predicciones de entrenamiento con NaN en las primeras posiciones (lookback)
        df[col_pred_train] = np.nan
        df.loc[self.lookback:self.lookback + len(train_preds) - 1, col_pred_train] = train_preds.flatten()

        # Agregar columna de predicciones de prueba, comenzando después de predicciones de entrenamiento + 2 * lookback
        start_test = len(train_preds) + 2 * self.lookback
        df[col_pred_test] = np.nan
        df.loc[start_test:start_test + len(test_preds) - 1, col_pred_test] = test_preds.flatten()

        return df

### Obtener predicciones de Modelos

#### Consumos de Combustible

##### Gasolina Superior

In [6]:
lstm_consumo_super = LSTM_Model(
    hidden_layers=55,
    num_layers=1,
    lookback=3,
    n_epochs=100,
    test_size=36,
    dataframe=consumos,
    target_column="Gasolina Superior"
)
consumos = lstm_consumo_super.updated_data

**NOTA**: El número de predicciones que faltan (valores **NaN**) se debe a que el modelo LSTM utiliza ventanas de tiempo (definidas por `lookback`) para hacer las predicciones, lo que implica que para cada conjunto de datos (entrenamiento y prueba), el número total de predicciones será menor que el total de registros en el conjunto original.

In [7]:
consumos.head()

,Fecha,Gasolina Regular,Gasolina Superior,Diesel,pred_train_Gasolina Superior,pred_test_Gasolina Superior
0,2000-01-01,202645.20,308156.82,634667.06,NaN,NaN
1,2000-02-01,205530.96,307766.31,642380.66,NaN,NaN
2,2000-03-01,229499.56,331910.29,699807.25,NaN,NaN
3,2000-04-01,210680.40,315648.08,586803.98,330589.46875,NaN
4,2000-05-01,208164.34,319667.97,656948.20,330105.53125,NaN


In [8]:
consumos.tail()

,Fecha,Gasolina Regular,Gasolina Superior,Diesel,pred_train_Gasolina Superior,pred_test_Gasolina Superior
288,2024-01-01,830708.13,658083.66,1359012.49,NaN,642059.3125
289,2024-02-01,818740.16,654059.60,1340174.42,NaN,643978.2500
290,2024-03-01,870771.70,671997.05,1393324.52,NaN,637729.8125
291,2024-04-01,847353.15,633520.57,1428143.44,NaN,640871.1875
292,2024-05-01,894533.14,692427.94,1401052.37,NaN,630325.6250


##### Gasolina Regular

##### Diesel